In [ ]:
%%bash
pip install pyautogui

In [ ]:
import pyautogui
import math
import time

# 圆周运动的参数
radius = 100  # 圆的半径
center_x, center_y = pyautogui.size().width // 2, pyautogui.size().height // 2  # 屏幕中心点
angle_step = 5  # 每步移动的角度增量，单位是度

# 进入无限循环
try:
    while True:
        for angle in range(0, 360, angle_step):
            # 计算当前角度的弧度值
            radian = math.radians(angle)
            # 计算新坐标
            x = center_x + radius * math.cos(radian)
            y = center_y + radius * math.sin(radian)
            # 移动鼠标
            pyautogui.moveTo(x, y)
            # 可以调整时间间隔来控制运动速度
            time.sleep(0.01)
except KeyboardInterrupt:
    print('圆周运动已停止。')

In [ ]:
%%bash
pip install pyautogui pynput

In [ ]:
import tkinter as tk
import pyautogui
import math
import threading
import time
from pynput import keyboard

class MouseCircleApp:
    def __init__(self, root):
        self.root = root
        self.root.title('Mouse Circle App')
        self.root.geometry('300x100')
        self.root.attributes('-topmost', True)

        self.running = False
        self.thread = None

        self.button = tk.Button(root, text='Start Circle', command=self.toggle_circle)
        self.button.pack(expand=True)
        
        self.listener = keyboard.Listener(on_press=self.on_key_press)
        self.listener.start()

    def toggle_circle(self):
        if not self.running:
            self.running = True
            self.button.config(text='Stop Circle')
            self.thread = threading.Thread(target=self.start_circle)
            self.thread.start()
        else:
            self.running = False
            self.button.config(text='Start Circle')

    def start_circle(self):
        radius = 100
        center_x, center_y = pyautogui.size().width // 2, pyautogui.size().height // 2
        angle_step = 5

        while self.running:
            for angle in range(0, 360, angle_step):
                if not self.running:
                    break
                radian = math.radians(angle)
                x = center_x + radius * math.cos(radian)
                y = center_y + radius * math.sin(radian)
                pyautogui.moveTo(x, y)
                time.sleep(0.01)

    def on_key_press(self, key):
        if key == keyboard.Key.esc:
            self.running = False
            self.button.config(text='Start Circle')
            if self.thread and self.thread.is_alive():
                self.thread.join()

if __name__ == '__main__':
    root = tk.Tk()
    app = MouseCircleApp(root)
    root.mainloop()

In [ ]:
%%bash
pyinstaller --onefile --noconsole circle.py

---
`awake.py`

使用 `ctypes` 库来调用 Windows API `SetThreadExecutionState` 防止系统进入睡眠或关闭显示器。

In [ ]:
%%bash
pip install pystray pillow

In [ ]:
import ctypes
import os
import pyautogui
import math
import time
import threading
import tkinter as tk
from pystray import Icon, MenuItem, Menu
from PIL import Image
from pynput import keyboard

# Constants for preventing sleep using SetThreadExecutionState API
ES_CONTINUOUS = 0x80000000  # Ensure the state is reset when the program exits
ES_DISPLAY_REQUIRED = 0x00000002  # Prevent the display from turning off
ES_SYSTEM_REQUIRED = 0x00000001  # Prevent the system from entering sleep

# Global variables for mouse circle functionality
running = False
circle_thread = None
button = None

def prevent_sleep():
    # Call the Windows API to prevent sleep and display off
    ctypes.windll.kernel32.SetThreadExecutionState(
        ES_CONTINUOUS | ES_DISPLAY_REQUIRED | ES_SYSTEM_REQUIRED
    )

def allow_sleep():
    # Reset the system to allow sleep and display off
    ctypes.windll.kernel32.SetThreadExecutionState(ES_CONTINUOUS)

def keep_awake():
    while True:
        # Function to simulate user activity to keep the system awake
        prevent_sleep()
        pyautogui.move(1, 0)  # Slight mouse movement to prevent idle state
        pyautogui.move(-1, 0)  # Move back to original position
        time.sleep(60 * 3)  # Sleep for 3 minutes before repeating

def on_quit(icon, root):
    # Handle the quit action from the system tray menu
    icon.stop()  # Stop the tray icon
    allow_sleep()  # Allow the system to sleep again
    root.quit()  # Quit the Tkinter main loop
    os._exit(0)  # Ensure the process exits completely

def show_window(root):
    # Show the main application window
    root.deiconify()
    root.attributes('-topmost', True)  # Bring the window to the front

def hide_window(root):
    # Hide the main application window
    root.withdraw()

def create_tray_icon(root):
    # Create a system tray icon with a menu
    image = Image.new('RGB', (64, 64), color=(0, 0, 0))  # Simple black icon
    menu = Menu(
        MenuItem('Open Window', lambda icon, item: show_window(root), default=True),
        MenuItem('Exit', lambda icon, item: on_quit(icon, root))
    )
    return Icon('Awake', image, 'Keep Awake', menu)

def toggle_circle(button):
    global running, circle_thread
    if not running:
        button.config(text='Stop Circle')
        # Start the circle motion
        running = True
        circle_thread = threading.Thread(target=start_circle)
        circle_thread.start()
    else:
        button.config(text='Start Circle')
        # Stop the circle motion
        running = False
        if circle_thread and circle_thread.is_alive():
            circle_thread.join()

def start_circle():
    global running
    radius = 100
    center_x, center_y = pyautogui.size().width // 2, pyautogui.size().height // 2
    angle_step = 5

    while running:
        for angle in range(0, 360, angle_step):
            if not running:
                break
            radian = math.radians(angle)
            x = center_x + radius * math.cos(radian)
            y = center_y + radius * math.sin(radian)
            pyautogui.moveTo(x, y)
            time.sleep(0.01)

def on_key_press(key):
    global button
    if key == keyboard.Key.esc:
        toggle_circle(button)

def main():
    # Set up the main Tkinter window
    root = tk.Tk()
    root.title('Awake & Circle')
    root.geometry('350x100')
    root.resizable(False, False)  # Fix the window size
    root.attributes('-topmost', True)  # Keep the window on top
    root.protocol('WM_DELETE_WINDOW', lambda: hide_window(root))  # Hide on close

    # Create the mouse circle button in the main window
    global button
    button = tk.Button(root, text='Start Circle', command=lambda: toggle_circle(button))
    button.pack(expand=True)

    # Start a background thread to keep the system awake
    threading.Thread(target=keep_awake, daemon=True).start()

    # Start the keyboard listener
    listener = keyboard.Listener(on_press=on_key_press)
    listener.start()

    # Create and run the system tray icon
    icon = create_tray_icon(root)
    threading.Thread(target=icon.run, daemon=True).start()

    # Run the Tkinter main loop
    root.mainloop()

if __name__ == '__main__':
    main()

说明：

1. `SetThreadExecutionState`：
   - `ES_CONTINUOUS`：保持该设置直到下一次调用。
   - `ES_DISPLAY_REQUIRED`：防止屏幕关闭。
   - `ES_SYSTEM_REQUIRED`：防止系统进入睡眠。

2. **防止和恢复**：
   - `prevent_sleep()`：调用后，系统不会进入睡眠或关闭显示器。
   - `allow_sleep()`：调用后恢复系统默认的电源管理设置。

3. **运行和停止**：
   - 运行脚本后，它会每分钟保持系统活跃，防止自动熄屏和锁屏。
   - 使用 `Esc` 和菜单选项 `Exit` 停止脚本运行，并恢复系统默认设置。

注意：
- 这个脚本仅在 Windows 操作系统上有效。
- `SetThreadExecutionState` 是 Windows 提供的 API，用于管理电源和系统状态。务必小心使用，防止意外阻止系统进入节能状态。

In [ ]:
%%bash
pyinstaller --onefile --noconsole awake.py

---
`setup.py`

In [ ]:
from setuptools import setup
from mypyc.build import mypycify

setup(
    name='awake',
    ext_modules=mypycify(['awake.py']),
    zip_safe=False,
)

In [ ]:
%%bash
pip install mypy

In [ ]:
%%bash
python setup.py build_ext --inplace

---
`main.py`

In [ ]:
import awake

if __name__ == '__main__':
    awake.main()  # 调用编译后的模块的 main 函数

In [ ]:
%%bash
pyinstaller --onefile main.py

---
WMI 服务管理与清理自动化

In [ ]:
import os
import shutil
import subprocess

def stop_and_disable_service(service_name):
    '''停止并禁用指定的服务'''
    try:
        # 停止服务
        subprocess.run(['sc', 'stop', service_name], check=True, text=True)
        print(f'服务已停止：{service_name}')

        # 禁用服务
        subprocess.run(['sc', 'config', service_name, 'start=', 'disabled'], check=True, text=True)
        print(f'服务已禁用：{service_name}')
    except subprocess.CalledProcessError as e:
        print(f'操作服务 {service_name} 时发生错误：{e}')

def clear_folder_contents(folder_path):
    '''清理文件夹下的所有内容'''
    if os.path.exists(folder_path):
        try:
            # 删除文件夹下的所有文件和子目录
            for item in os.listdir(folder_path):
                item_path = os.path.join(folder_path, item)
                if os.path.isdir(item_path):
                    shutil.rmtree(item_path)
                else:
                    os.remove(item_path)
            print(f'成功清理文件夹内容：{folder_path}')
        except Exception as e:
            print(f'清理文件夹 {folder_path} 时发生错误：{e}')
    else:
        print(f'路径不存在：{folder_path}')

if __name__ == '__main__':
    # 停止并禁用服务
    stop_and_disable_service('winmgmt')
    stop_and_disable_service('CcmExec')

    # 定义文件夹路径
    wbem_base_path = r'C:\\Windows\\System32\\wbem'
    auto_recover_path = os.path.join(wbem_base_path, 'AutoRecover')
    repository_path = os.path.join(wbem_base_path, 'Repository')
    remote_control_path = r'C:\\Windows\\CCM\\RemCtrl'

    # 清理 wbem/AutoRecover 和 wbem/Repository 文件夹
    clear_folder_contents(auto_recover_path)
    clear_folder_contents(repository_path)
    # 清理 CCM/RemCtrl 文件夹
    clear_folder_contents(remote_control_path)